In [ ]:
from bs4 import BeautifulSoup
import base64
import wget
import requests
import unicodecsv as csv

In [ ]:
#URL inicial
url = 'https://elcomercio.pe/archivo/todas/2019-09-12'

In [ ]:
# Obtener Links
content = requests.get(url).content

In [ ]:
soup = BeautifulSoup(content,'lxml') # choose lxml parser

In [ ]:
# Ubicar tags : <div class="toc">
articles = soup.findAll('article', {'class' : 'section-flow'}) # id="toc" also works

In [ ]:
total_articles = []
cont = 0
for article in articles:
    url_article = 'https://elcomercio.pe' + article.find('a', {'class': 'page-link'})['href']
    content_article = requests.get(url_article).content
    soup_article = BeautifulSoup(content_article,'lxml')
    
    title = soup_article.find('h1', {'class': 'news-title'}).text
    summary = soup_article.find('h2', {'class': 'news-summary'}).text
    category = soup_article.find('h3', {'class': 'news-category fija'}).find('a').text
    author = soup_article.find('div', {'class': 'news-author-date'}).find('a').find('span').text
    date = soup_article.find('div', {'class': 'news-author-date'}).find('time')['datetime']
    text_content = soup_article.find('div', {'class': 'news-text-content'}).findAll('p', {'class': 'parrafo'})
    content_article = []
    for text in text_content:    
        content_article.append(text.text)
    
    tags_article = soup_article.findAll('h2', {'class': 'tags-item'})
    tags = []
    for tag in tags_article:
        tags.append(tag.text)
        
    total_articles.extend([
        {
            'Titulo':str(title), 
            'Resumen': str(summary), 
            'Categoria':str(category), 
            'Autor':str(author),
            'Fecha':str(date),
            'Contenido':str(content_article),
            'Etiquetas':str(tags)
        }
    ])
    with open('el_comercio.csv', 'wb') as f:
            dict_writer = csv.DictWriter(f,['Titulo','Resumen','Categoria', 'Autor', 'Fecha', 'Contenido', 'Etiquetas'],encoding='utf-8')
            dict_writer.writeheader()
            dict_writer.writerows(total_articles)
    
    cont = cont + 1
    
    print('---------------------------------------------------------------  ' + str(cont*len(articles)/100) + "%")